# Writing Agents


In [25]:
import os
from openai import OpenAI
from pydantic import BaseModel
from enum import Enum
from typing import List
import pprint
import create_doc



client = OpenAI()

# MODEL = "gpt-4o-2024-08-06"
MODEL = "gpt-4o-mini-2024-07-18"
SEARCH_MODEL = "sonar"

user_prompt = "Write a paper arguing why AI in the workplace won't replace humans"

## Research Section

This section implements a multi-agent research orchestration system designed to automate academic information gathering and synthesis. The pipeline consists of two specialized agents working in sequence:

1. **Research Planning Agent**
- Constructs a list of 3-5 search queries based on the topic and the user's prompt
- Outputs structured query plans that maximize coverage and relevance

2. **Search Execution Agent**
- Uses Perplexity API to execute the search queries
- Retrieves and ranks authoritative sources based on relevance scores



In [26]:
research_planner_system_prompt = """
You are a research assistant that helps with the planning of a research paper. Given a topic, you will provide a list of 3-5 searches that will provide helpful information for the paper.
"""

class ResearchPlan(BaseModel):
  searches: List[str]

research_planner_response = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": research_planner_system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    response_format=ResearchPlan,
)

pprint.pprint(research_planner_response.choices[0].message.parsed.searches)



['impact of AI on employment in various industries',
 'AI augmentation vs. AI replacement in the workplace',
 'human skills that AI cannot replicate',
 'case studies on AI and job transformation',
 'future job market predictions with AI integration']


In [27]:
perplexity_api_key = os.environ["PERPLEXITY_API_KEY"]
perplexity = OpenAI(api_key=perplexity_api_key, base_url="https://api.perplexity.ai")


In [28]:
research_system_prompt = """
You are a highly capable research assistant specializing in academic research and providing scholarly, authoritative, and credible sources. Your primary goal is to assist someone writing an argumentative paper by identifying and summarizing the most relevant and reliable sources available on the internet. 

Focus on delivering:
1. **Scholarly Articles**: Peer-reviewed journal articles, conference papers, and research studies from reputable academic publishers (e.g., Springer, IEEE, Elsevier, JSTOR).
2. **Official Reports**: Publications from government agencies, international organizations, and established think tanks.
3. **Credible Websites**: Pages from university domains (.edu), respected research organizations, and verified expert authors.
4. **Primary Sources**: Original works, raw data, or foundational theories when relevant.

When researching, ensure that:
- **Relevance**: The sources directly address the central idea or argument of the paper.
- **Credibility**: Prioritize sources with strong evidence, citations, and authoritative authorship.
- **Diversity**: Offer a range of perspectives or insights to enrich the paper’s argumentation.
- **Accessibility**: If possible, prioritize sources that are freely available or provide summaries for sources behind paywalls.

For each source:
- Provide the **title**, **author(s)**, **publication date**, and **URL**.
- Summarize the key findings, arguments, or data presented in the source in 2-3 sentences.
- Indicate the **type of source** (e.g., journal article, government report, book chapter).
- Optionally include the **citation format** (e.g., APA, MLA) to save time for the writer.

Your tone should be concise, professional, and focused on providing value to the writer.

Here’s an example response structure:
1. **Source**: [Title] by [Author(s)] (Publication Date)  
   - **Type**: [Journal article/Report/etc.]  
   - **Summary**: [Brief summary of the content and its relevance.]  
   - **URL**: [Link]  
   - **Citation**: [Optional formatted citation]

Always aim for depth and accuracy to help the writer build a well-informed and persuasive argument.
"""

In [21]:
research_responses = {}
citations = []

for search in research_planner_response.choices[0].message.parsed.searches:

    messages = [
        {
            "role": "system",
            "content": research_system_prompt,
        },
        {   
            "role": "user",
            "content": search,
        },
    ]

    research_response = perplexity.chat.completions.create(
        model=SEARCH_MODEL,
        messages=messages,
    )

    research_responses[search] = research_response.choices[0].message.content
    citations += research_response.citations


In [ ]:
# pprint.pprint(research_responses)
# pprint.pprint(citations)

## Writing Section
Writing Plan Construction with Structured Schemas

This section implements a multi-agent workflow for constructing structured argumentative papers:

1. An orchestrator agent defines the high-level paper structure and requirements using 
   strictly typed schemas for paragraph types, transitions, and argument flow

2. Worker agents are assigned specific paragraph generation tasks, each operating within
   defined schema constraints to ensure consistency

The workflow follows a hierarchical pattern:
- Orchestrator: Plans overall structure, assigns tasks, validates coherence
- Workers: Generate paragraph content following schema specifications
- Validators: Ensure schema compliance and argument strength

This structured approach enables systematic paper construction while maintaining
argument quality through schema-enforced constraints.


In [ ]:
system_prompt = """
You are an expert author tasked with crafting a high-quality argumentative paper on a given topic, designed to resemble a compelling newspaper opinion piece. 
Your paper must follow a clear, logical structure and maintain a strong, engaging flow between paragraphs, avoiding redundancy while building a persuasive and cohesive argument. 

The paper must revolve around a well-defined thesis statement, which serves as the backbone of the argument. Every paragraph should explicitly or implicitly support the thesis, creating a unified and focused narrative.

### Instructions:
1. **Structure and Organization**:
   - Begin the paper by clearly defining the thesis statement, ensuring it is specific, debatable, and sets the tone for the argument.
   - The paper should include an outline with a compelling title and a sequence of well-structured paragraphs.
   - Each paragraph must serve a distinct purpose, explicitly or implicitly supporting the thesis and connecting seamlessly to the next for a natural progression of ideas.

2. **For Each Paragraph**:
   - Assign a sequential number.
   - Provide a descriptive name reflecting its role in the paper.
   - Specify the paragraph type using the provided taxonomy.
   - Include a concise, focused prompt designed to guide another LLM in generating the content for that paragraph.
   - Clearly articulate how the paragraph relates to and supports the thesis.

3. **Types of Paragraphs** (with descriptions):
   - **Introduction**: Grabs attention, introduces the topic, presents the thesis or main argument, and outlines the structure of the paper.
   - **Expository**: Explains key concepts, evidence, or background information essential for understanding the thesis.
   - **Argumentative**: Advances a specific claim supported by evidence and analysis that directly supports the thesis.
   - **Comparative**: Examines similarities or differences between two ideas, sources, or perspectives to highlight aspects that reinforce the thesis.
   - **Synthesizing**: Connects multiple sources, arguments, or ideas to build a unified perspective and strengthen the thesis.
   - **Counterargument**: Acknowledges opposing viewpoints, refutes them with reasoning and evidence, and reinforces the thesis.
   - **Transitional**: Bridges ideas or sections to ensure a smooth flow and logical progression while maintaining focus on the thesis.
   - **Analytical**: Delves into the implications, significance, or deeper meaning of evidence or a source in the context of the thesis.
   - **Evaluative**: Critiques a source, argument, or perspective, assessing its credibility, strengths, and weaknesses in relation to the thesis.
   - **Conclusion**: Summarizes the main arguments, reinforces the thesis, and provides a compelling closing statement or call to action.

4. **Writing Style**:
   - Use a concise, persuasive tone that mirrors the style of a top-tier opinion piece.
   - Avoid redundancy by ensuring each paragraph introduces new insights or ideas that contribute to and reinforce the thesis.
   - Prioritize logical flow between paragraphs to guide the reader smoothly through the argument while maintaining a consistent focus on the thesis.

5. **Output**:
   - The output should include a detailed outline of the paper with a dedicated section for the thesis statement.
   - Ensure each paragraph prompt is precise, relevant, and explicitly tied to the thesis.
   - Include space in the structured output to define the thesis statement and demonstrate how each paragraph supports it.
"""

class ParagraphType(str, Enum):
  introduction = "introduction"
  expository = "expository"
  argumentative = "argumentative"
  comparative = "comparative"
  synthesizing = "synthesizing"
  counterargument = "counterargument"
  transitional = "transitional"
  analytical = "analytical"
  evaluative = "evaluative"
  conclusion = "conclusion"

class Paragraph(BaseModel):
  number: int
  name: str
  paragraphType: ParagraphType
  prompt: str
  
class PaperStructure(BaseModel):
  title: str
  thesis: str
  paragraphs: List[Paragraph]

# master thread
response = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
        ],
    response_format=PaperStructure,
)

pprint.pprint(response.choices[0].message.parsed)

PaperStructure(title='The Imperative Role of Facial Recognition Technology in Airport Security', thesis='Facial recognition technology is a crucial tool for enhancing airport security, as it significantly improves threat detection efficiency, ensures quick and accurate passenger identification, and strengthens the overall safety of air travel.', paragraphs=[Paragraph(number=1, name='Introduction to Facial Recognition in Airports', paragraphType=<ParagraphType.introduction: 'introduction'>, prompt='Introduce the significance of airport security in the modern world, highlighting the ongoing challenges faced by traditional security measures. Present the thesis that facial recognition technology is essential for enhancing airport security, offering efficiency, accuracy, and safety improvements. Outline the structure of the paper, setting the stage for discussing the benefits and addressing potential concerns of this technology in security contexts with a focus on airports. This introductio

In [43]:
print (f"Title: {response.choices[0].message.parsed.title}")
print (f"Thesis: {response.choices[0].message.parsed.thesis}")
for paragraph in response.choices[0].message.parsed.paragraphs:
  print(f"{paragraph.number}.   {paragraph.name} ({paragraph.paragraphType.value})")
#   print(f"Prompt: {paragraph.prompt}")
#   print("\n")

Title: The Imperative Role of Facial Recognition Technology in Airport Security
Thesis: Facial recognition technology is a crucial tool for enhancing airport security, as it significantly improves threat detection efficiency, ensures quick and accurate passenger identification, and strengthens the overall safety of air travel.
1.   Introduction to Facial Recognition in Airports (introduction)
2.   Understanding Facial Recognition Technology (expository)
3.   Enhancing Efficiency in Threat Detection (argumentative)
4.   Accuracy in Passenger Identification (argumentative)
5.   Comparative Analysis with Traditional Security Methods (comparative)
6.   Addressing Privacy Concerns (counterargument)
7.   Seamless Integration and Future Prospects (synthesizing)
8.   Implications for Travelers and Airport Operations (analytical)
9.   Reinforcing Global Aviation Security (evaluative)
10.   Conclusion and Call to Action (conclusion)


### Writing paragraphs

In [ ]:
system_prompt = """
You are an expert writer tasked with crafting a single, high-quality paragraph for an argumentative paper.

You will be provided with the full structure of the paper, including the names and types of each paragraph in order, as well as a prompt defining the focus of the paragraph you are writing and the thesis of the paper.

Your writing must be concise, meaningful, and directly tied to the thesis while ensuring smooth transitions between paragraphs and sections.

### Instructions:

1. **Understand the Context**:

   - Review the full structure of the paper to understand the flow and relationships between paragraphs.
   - Identify the role of the paragraph you are crafting within the structure and how it connects to the previous and next paragraphs.
   - Use the provided prompt to craft a focused, purposeful paragraph that aligns with the thesis and contributes to the overall logical progression of the paper.

2. **Paragraph Types**:

   - **Introduction**: Hook the reader, introduce the topic, present the thesis, and briefly outline the paper's key arguments. Ensure this paragraph establishes a strong foundation for the paper's flow.
   - **Expository**: Provide essential context or explain key evidence directly related to the thesis. Connect the context to the prior argument and set up the next paragraph.
   - **Argumentative**: Present a strong, specific claim backed by evidence that directly supports the thesis. Conclude by preparing the reader for the next argument or evidence.
   - **Comparative**: Analyze similarities or differences to highlight aspects that strengthen the thesis. Smoothly connect comparisons to prior and forthcoming paragraphs.
   - **Synthesizing**: Combine ideas or sources to form a cohesive argument that advances the thesis. Tie synthesized ideas to the preceding discussion and suggest implications for the next section.
   - **Counterargument**: Address and refute opposing views with clear evidence and reasoning. Transition smoothly from prior points and guide the reader back to the thesis.
   - **Transitional**: Connect ideas or sections to ensure smooth, logical progression while maintaining focus on the thesis. Serve as a bridge that reinforces continuity and introduces the next section.
   - **Analytical**: Explore the deeper implications or significance of evidence in relation to the thesis. Link implications to prior evidence and analysis and set up subsequent arguments.
   - **Evaluative**: Critique a source or argument, focusing on its relevance and impact on the thesis. Ensure the critique builds on prior evidence and analysis and transitions to the next key point.
   - **Conclusion**: Summarize key points, restate the thesis, and provide a strong closing insight or call to action. The final sentence should unify the discussion and leave a lasting impression.

3. **Writing Style**:

   - Use clear, direct language that conveys meaningful content without unnecessary words. Don't use overly complex language.
   - Avoid redundancy and focus on presenting new insights or advancing the argument.
   - Maintain a logical flow that ties each paragraph to the thesis and ensures smooth progression between ideas and sections.
   - Include in-text citations in APA format (Author, Year) when referencing sources or evidence.

4. **Paragraph Structure**:

   - Begin with a topic sentence that establishes the paragraph's main idea and links it to the previous paragraph.
   - Support the idea with concise evidence, analysis, or reasoning, including appropriate in-text citations for all evidence and claims from sources.
   - End with a sentence that reinforces the thesis and transitions logically to the next section.

5. **Additional Guidance for Full Paper Structure**:

   - Refer to the names and types of each paragraph to understand their individual roles and how they contribute to the overall argument.
   - Ensure each paragraph builds on the ideas established in previous paragraphs and sets up the next for a cohesive narrative.
   - Use transitional phrases and logical connections to maintain smooth and seamless progression.
   - If you can reasonably assume that an abbreviation or idea has been defined in a previous paragraph or in the thesis, you should not redefine it and can use it as needed.
   - Consistently cite sources using APA format in-text citations.

6. **Output**:

   - Write a single paragraph of 150–250 words unless specified otherwise.
   - Ensure the paragraph is concise, precise, and ready to be part of the larger argument.
   - Explicitly address the prompt, connect to the thesis in a meaningful way, and ensure smooth transitions from and to other paragraphs.
   - Include appropriate in-text citations for all evidence and claims from sources.

"""




responses = []
thesis = response.choices[0].message.parsed.thesis
structure = "".join([f"{i}. {paragraph.name} ({paragraph.paragraphType.value})\n" for i, paragraph in enumerate(response.choices[0].message.parsed.paragraphs)])
research = "\n".join(research_responses)

for paragraph in response.choices[0].message.parsed.paragraphs:
   user_prompt = f"""Name: <name>{paragraph.name}</name>
      Type: <type>{paragraph.paragraphType.value}</type>
      Prompt: <prompt>{paragraph.prompt}</prompt>
      Thesis: <thesis>{thesis}</thesis>
      Paragraph Structure: <structure>{structure}</structure>
      Research: <research>{research_responses}</research>"""

   r = client.beta.chat.completions.parse(
         model=MODEL,
         messages=[
               {"role": "system", "content": system_prompt},
               {"role": "user", "content": user_prompt}
               ]
      )
   responses.append(r)

print(responses[0])

ParsedChatCompletion[NoneType](id='chatcmpl-ApOSfsWPdXrmzsRrEP7mrRMd1XjuT', choices=[ParsedChoice[NoneType](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[NoneType](content="In today's fast-paced and interconnected world, airport security has become a cornerstone of global transportation, underpinning the safety and confidence of millions of travelers every day. However, traditional security measures, often reliant on physical screenings and human oversight, face mounting challenges in effectively managing the increasing volume of passengers while ensuring robust threat detection. Such methods can be time-consuming, prone to human error, and sometimes inadequate in the face of sophisticated threats. Injecting efficiency and precision into this arena is thus paramount, which is where facial recognition technology comes into play. This paper argues that facial recognition technology is a crucial tool for enhancing airport security, as it significantly i

In [45]:
paragraphs = [r.choices[0].message.content for r in responses]
pprint.pprint(paragraphs[3])

('Facial recognition technology ensures greater accuracy in passenger '
 'identification than manual checks, providing a robust defense against human '
 'error and inefficiencies. Unlike traditional identification methods reliant '
 'on human observation and document verification, facial recognition harnesses '
 'advanced algorithms to analyze biometric data, drastically reducing the '
 'likelihood of oversight or misidentification. This technological advantage '
 'is evidenced by case studies conducted at major airports, such as the trial '
 'at Atlanta’s Hartsfield-Jackson Airport, where facial recognition improved '
 'identification accuracy from the typical 85-90% associated with manual '
 'checks to an impressive 98%. By accurately matching passengers against '
 'comprehensive databases, facial recognition swiftly and reliably identifies '
 'individuals flagged as potential threats or listed on security watchlists. '
 'This heightened precision in detecting suspicious individuals 

## Exporting to Google Docs

In [23]:
# Sample data (replace with your own references)
title = response.choices[0].message.parsed.title

create_doc.create_document(paragraphs, thesis, title, citations)

NameError: name 'response' is not defined